In [1]:
import requests
import lxml.html
import pandas as pd
import re

In [2]:
res = requests.get("https://news.naver.com/section/101")
res

<Response [200]>

In [3]:
root = lxml.html.fromstring(res.text)
root

<Element html at 0x22c787b6770>

In [4]:
urls = []
for link in root.cssselect("a.sa_text_title"):
    urls.append(link.attrib["href"])

In [5]:
print(urls[0])
print(len(urls))

https://n.news.naver.com/mnews/article/001/0014686923
46


In [6]:
titles = []
articles = []
i = 0

for u in urls:
    if not u.startswith("http"):
        continue
        
    res = requests.get(u)
    article_root = lxml.html.fromstring(res.text)
    
    # 기존 오류는 url에 접속하여 해당 기사로 이동 후 원하는 태그 영역의 데이터를 불러오는 것인데, 
    # 뉴스 사이트마다 태크에 사용한 id혹은 class명이 달라서 오류가 발생했습니다.
    # 이를 해결하기 위해 try except문을 사용하여 에러나는 부분은 넘어가게 만들었습니다.
    try:
        title_elements = article_root.cssselect('h2#title_area').pop()
        body_elements = article_root.cssselect('article#dic_area').pop()
    except:
        continue
        
    title = title_elements.text_content()
    
    content = body_elements.text_content()
    content = re.sub(r'[\n\t]+', '', content)
        
    titles.append(title)
    articles.append(content)

In [7]:
df = pd.DataFrame({"title":titles,"article":articles})
df.head()

,title,article
0,CJ대한통운 한국사업부문 신임 대표에 윤진 FT본부장,윤진 CJ대한통운 한국사업부문 신임 대표[CJ대한통운 제공. 재판매 및 DB 금지]...
1,금투협-운용사 밸류업 간담회…상장사 기업가치 제고 지원,금융당국이 ‘기업 밸류업’ 불씨 살리기에 안간힘을 쓰고 있는 가운데 금융투자협회도 ...
2,"보험 순익 양극화…삼성, 홀로 '7020억' vs 현대·KB, 합쳐서 '7695억'",보험사 순이익 양극화삼성화재가 올 1분기 당기순이이익 1위 자리를 공고히했다. 사진...
3,"저축은행, 부실채권 정리 위한 2000억원 자체펀드 조성",캠코매각 추진과 약 2000억원 규모 자체펀드 조성 추진저축은행중앙회가 저축은행의 ...
4,작년 에코프로 ‘숏스퀴즈’ 떠오르네…“美 게임스톱 개미 ‘선빵’에 공매도 세력 하루...,[게티이미지뱅크][헤럴드경제=신동윤 기자] 지난 3년전 대표적인 ‘밈(Meme) 주...


In [8]:
import os

# 파일명
file_name = "ai_기사.xlsx"

# 파일이 이미 존재하는지 확인하고 삭제
if os.path.exists(file_name):
    os.remove(file_name)

# Excel 파일 생성
df.to_excel(file_name)